 ### Введение, примеры задач, бизнес- и ML-метрики

##### 1. Сравните метрики hit_rate@k, precision@k. Какую метрику использовать предпочтительно и почему? Приведите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?

Предпочтительно использовать precision@k, т.к. он показывает долю попавших рекомендаций в выборку, а hit_rate@k 
просто попал ли хоть один рекомендованный в выборку или нет. Метрику hit_rate@k наверное можно использовать в так называемый "холодный поиск" когда сосвем мало информации на начальной стадии, чтобы отсеивать выбрки если данная метрика нулевая

##### 2. В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм? Какую функцию можно использовать вместо логарифма? Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в логарифм

С точки зрения ранжирования позиции в начале списка отличаются гораздо сильнее, чем позиции в его конце. Так, в случае поискового движка между позициями 1 и 11 целая пропасть (лишь в нескольких случаях из ста пользователь заходит дальшей первой страницы поисковой выдачи), а между позициями 101 и 111 особой разницы нет — до них мало кто доходит. logloss

##### 3. Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики и чем являются интеракции, чтобы она могла быть использована)

Метрики на основе ранговой корреляции: коэффициенты Спирмена и Кендэлла.
Метрики на основе каскадной модели поведения: Expected reciprocal rank и PFound

##### 4. boughted = [1, 3, 5, 7, 9, 11]  recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]  Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, RR@8, ERR@8 (expected reciprocal rank)  Сначала поссчитайет "руками", а затем подтвердите рассчеты с помощью функций (некоторые были на семинаре, некоторые нужно будут написать самостоятельно)

In [3]:
import pandas as pd
import numpy as np

In [4]:
boughted = [1, 3, 5, 7, 9, 11] 
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]

$precison@8 = \frac{0 + 0 + 1 + 1 + 1 + 1}{1 + 1 + 1 + 1 + 1 + 1 + 1 + 1} = 50\%$

In [5]:
def precision_at_k(recommended_list, bought_list, k=8):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    print(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    print(flags)
    
    print(flags.sum())
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [6]:
precision_at_k(recommended, boughted, k=8)

[ 2  5  7  4 11  9  8 10]
[False False  True  True  True  True]
4


0.5

In [7]:
precision_at_k(recommended, boughted, k=3)

[2 5 7]
[False False  True  True False False]
2


0.6666666666666666

$recall@8 = \frac{0 + 0 + 1 + 1 + 1 + 1}{1 + 1 + 1 + 1 + 1 + 1} = 66,7\%$

In [8]:
def recall_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    print(flags)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

In [9]:
recall_at_k(recommended, boughted, k=8)

[False False  True  True  True  True]


0.6666666666666666

$AP@8 = \frac{1}{8}(0 + 0 + \frac{1}{3} + \frac{2}{4} + \frac{3}{5} + \frac{4}{6}) = 0,2625$ 

In [23]:
def ap_k(recommended_list, bought_list, k=8):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    #recommended_list = recommended_list[:k]
    
    #flags = np.isin(bought_list, recommended_list)
    flags = np.isin(recommended_list, bought_list)
    print(flags)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(1, k+1): 
        if flags[i] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)
            sum_ += p_k
            print(sum_)
            
    result = sum_ / len(flags)
    
    return result

In [24]:
ap_k(recommended, boughted, k=8)

[False  True  True False  True  True False False False  True]
[2]
[False False False False False False]
0
0.0
[2 5]
[False False  True False False False]
1
0.5
[2 5 7 4]
[False False  True  True False False]
2
1.0
[ 2  5  7  4 11]
[False False  True  True False  True]
3
1.6


0.16

$NDCG@8 = \frac{\frac{1}{8}(\frac{0}{1}+\frac{1}{log(2)}+\frac{1}{log(3)}+\frac{0}{log(4)}+\frac{1}{log(5)}+\frac{1}{log(6)}+\frac{0}{log(7)}+\frac{0}{log(8)})}{\frac{1}{8}(\frac{1}{1}+\frac{1}{log(2)}+\frac{1}{log(3)}+\frac{1}{log(4)}+\frac{1}{log(5)}+\frac{1}{log(6)}+\frac{1}{log(7)}+\frac{1}{log(8)})}$